In [1]:
import os
from langchain.llms import HuggingFacePipeline
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.schema import Document
from transformers import LlamaTokenizer, LlamaForCausalLM, pipeline
from sentence_transformers import SentenceTransformer
import csv
import torch

documents = []
trainingSplit = ["federal communications commission", "terrorism", "pollution", "capitalism", "slavery", "prostitution", "pornography", "armenian genocide", "liberalism", "oil drilling", "censorship", "celtic revival", "law enforcement", "wikipedia", "fractional reserve banking", "esports", "prison", "world war i", "energy consumption", "prenatal care", 	"glyphosate", "autoerotic asphyxiation", "human evolution", "bullfighting", "fascism",	"opus dei", "sigmund freud", "depleted uranium", "1919 world series", "pedophilia", "mormonism"]

##the code below extracts the causal relations which are in the training dataset
with open('/bigwork/nhwpshaa/BigDataset/firstOrderEffect.csv', 'r') as file:
    csvfile = csv.DictReader(file)
    for row in csvfile:
        if row['cause'] in trainingSplit:
            content= f"Cause:{row['cause']} Effect:{row['effect']}" 
            documents.append(Document(page_content= content))
        
with open('/bigwork/nhwpshaa/BigDataset/cause.csv', 'r') as file:
    csvfile = csv.DictReader(file)
    for row in csvfile:
        if row['effect'] in trainingSplit:
            content= f"Cause:{row['cause']} Effect:{row['effect']}" 
            documents.append(Document(page_content= content))
            
    
with open('/bigwork/nhwpshaa/Fine-tuning/Alpaca Native/secondOrderData/trainSample.csv', 'r') as file:
    csvfile = csv.DictReader(file)
    for row in csvfile:
        content= f"Cause:{row['Cause']} Effect:{row['Effect']} Second Order Effect:{row['Second Order Effect']}" 
        documents.append(Document(page_content= content))
        
        

model = HuggingFaceEmbeddings(model_name='/bigwork/nhwpshaa/all-MiniLM-L6-v2') #Initialize embeddings
vectorDB = FAISS.from_documents(documents, model) #Initialize vector database
modelName = "/bigwork/nhwpshaa/alpaca-native"  # You can choose another model if you prefer
tokenizer = LlamaTokenizer.from_pretrained(modelName)
llm = LlamaForCausalLM.from_pretrained(modelName)
pipelineModel = pipeline("text-generation",
    model=llm,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
    min_length=500,
    max_length=900,
    do_sample=True,
    top_k=30,
    num_return_sequences=4,
    eos_token_id=tokenizer.eos_token_id,
    )
pipe = HuggingFacePipeline(pipeline=pipelineModel)
prompt = PromptTemplate(
    template="{context}\nAct as an expert debater and generate an essay of around 400 words arguing against the following topic. Structure the essay into well-organized paragraphs and provide evidence to support the claim \nTopic: {question}\nAnswer:",
    input_variables=["context", "question"]
)

chain = RetrievalQA.from_llm(
    llm=pipe,
    retriever=vectorDB.as_retriever(),
    prompt=prompt
)



question = "Countries should start using nuclear power as their main source of energy."
response = chain({"query": question})
print(f"Question: {question}\nAnswer: {response}")

question = "Restricting air travel is the only way to prevent air pollution."
response = chain({"query": question})
print(f"Question: {question}\nAnswer: {response}")

question = "censorship in modern society is quite necessary"
response = chain({"query": question})
print(f"Question: {question}\nAnswer: {response}")

question = "problem of illegal drugs can be solved by just legalizing all drugs"
response = chain({"query": question})
print(f"Question: {question}\nAnswer: {response}")

question = "holding the olympics wastes money which could be used for important issues"
response = chain({"query": question})
print(f"Question: {question}\nAnswer: {response}")


/home/nhwpshaa/.local/lib/python3.8/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

/home/nhwpshaa/.local/lib/python3.8/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFacePipeline`.
  warn_deprecated(
/home/nhwpshaa/.local/lib/python3.8/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


Question: Countries should start using nuclear power as their main source of energy.
Answer: {'query': 'Countries should start using nuclear power as their main source of energy.', 'result': 'Context:\nCause:capitalism Effect:nuclear war Second Order Effect:devastation of whole countries\n\nContext:\nCause:slavery Effect:crisis Second Order Effect:countries to the brink of nuclear exchange\n\nContext:\nCause:capitalism Effect:nuclear_war\n\nContext:\nCause:capitalism Effect:exploitation Second Order Effect:renewable and non-renewable energy resource development\nAct as an expert debater and generate an essay of around 400 words arguing against the following topic. Structure the essay into well-organized paragraphs and provide evidence to support the claim \nTopic: Countries should start using nuclear power as their main source of energy.\nAnswer: Nuclear power is not a viable option for countries to depend on for their main source of energy. It carries too much risk and is too expensiv

In [2]:
import os
from langchain.llms import HuggingFacePipeline
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.schema import Document
from transformers import LlamaTokenizer, LlamaForCausalLM, pipeline
from sentence_transformers import SentenceTransformer
import csv
import torch

documents = []
trainingSplit = ["federal communications commission", "terrorism", "pollution", "capitalism", "slavery", "prostitution", "pornography", "armenian genocide", "liberalism", "oil drilling", "censorship", "celtic revival", "law enforcement", "wikipedia", "fractional reserve banking", "esports", "prison", "world war i", "energy consumption", "prenatal care", 	"glyphosate", "autoerotic asphyxiation", "human evolution", "bullfighting", "fascism",	"opus dei", "sigmund freud", "depleted uranium", "1919 world series", "pedophilia", "mormonism"]

with open('/bigwork/nhwpshaa/BigDataset/firstOrderEffect.csv', 'r') as file:
    csvfile = csv.DictReader(file)
    for row in csvfile:
        if row['cause'] in trainingSplit:
            content= f"Cause:{row['cause']} Effect:{row['effect']}" 
            documents.append(Document(page_content= content))
        
with open('/bigwork/nhwpshaa/BigDataset/cause.csv', 'r') as file:
    csvfile = csv.DictReader(file)
    for row in csvfile:
        if row['effect'] in trainingSplit:
            content= f"Cause:{row['cause']} Effect:{row['effect']}" 
            documents.append(Document(page_content= content))
            
    
with open('/bigwork/nhwpshaa/Fine-tuning/Alpaca Native/secondOrderData/trainSample.csv', 'r') as file:
    csvfile = csv.DictReader(file)
    for row in csvfile:
        content= f"Cause:{row['Cause']} Effect:{row['Effect']} Second Order Effect:{row['Second Order Effect']}" 
        documents.append(Document(page_content= content))
        

model = HuggingFaceEmbeddings(model_name='/bigwork/nhwpshaa/all-MiniLM-L6-v2') #Initialize embeddings
vectorDB = FAISS.from_documents(documents, model) #Initialize vector database
modelName = "/bigwork/nhwpshaa/alpaca-native"  # You can choose another model if you prefer
tokenizer = LlamaTokenizer.from_pretrained(modelName)
llm = LlamaForCausalLM.from_pretrained(modelName)
pipelineModel = pipeline("text-generation",
    model=llm,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
    min_length=500,
    max_length=900,
    do_sample=True,
    top_k=30,
    num_return_sequences=4,
    eos_token_id=tokenizer.eos_token_id,
    )
pipe = HuggingFacePipeline(pipeline=pipelineModel)
prompt = PromptTemplate(
    template="{context}\nAct as an expert debater and generate an essay of around 400 words supporting the following topic. Structure the essay into well-organized paragraphs and provide evidence to support the claim \nTopic: {question}\nAnswer:",
    input_variables=["context", "question"]
)

chain = RetrievalQA.from_llm(
    llm=pipe,
    retriever=vectorDB.as_retriever(),
    prompt=prompt
)



question = "Tobacco and alcohol are drugs that cause addition and health problems."
response = chain({"query": question})
print(f"Question: {question}\nAnswer: {response}")

question = "Increasing the price of petrol is the best way to solve growing traffic and pollution problems."
response = chain({"query": question})
print(f"Question: {question}\nAnswer: {response}")

question = "if gun ownership were made illegal, most violent crime would be eliminated"
response = chain({"query": question})
print(f"Question: {question}\nAnswer: {response}")

question = "Telling the truth at all times is not essential."
response = chain({"query": question})
print(f"Question: {question}\nAnswer: {response}")

question = "government should increase taxes on alcoholic beverages"
response = chain({"query": question})
print(f"Question: {question}\nAnswer: {response}")


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Question: Tobacco and alcohol are drugs that cause addition and health problems.
Answer: {'query': 'Tobacco and alcohol are drugs that cause addition and health problems.', 'result': "Context:\nCause:pollution Effect:costs Second Order Effect:legal substances of tobacco\n\nContext:\nCause:pollution Effect:problems Second Order Effect:attachment to addictive drugs\n\nContext:\nCause:pollution Effect:health problems Second Order Effect:incorrect medications\n\nContext:\nCause:pollution Effect:injuries Second Order Effect:tobacco smoke exposure\nAct as an expert debater and generate an essay of around 400 words supporting the following topic. Structure the essay into well-organized paragraphs and provide evidence to support the claim \nTopic: Tobacco and alcohol are drugs that cause addition and health problems.\nAnswer: Although tobacco and alcohol are legal substances, they can have serious consequences on an individual's health and well-being. In some instances, tobacco and alcohol can